In [12]:
import json
import time

import subprocess
from datetime import datetime
import pytz

from pyspark.sql.functions import udf, to_timestamp, year, month, dayofmonth, hour, minute, second
from pyspark.sql.types import StructType, StructField, StringType

from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("DataProcessing").getOrCreate()

names_history = []

try:
    while True:

        command = "hadoop fs -ls /datalake/raw/stagging | awk '{print $NF}'"
        file_names = subprocess.check_output(command, shell=True).decode().split('\n')

        array_names = []

        for name in file_names:
            if ".parquet" in name:
                if name not in names_history:
                    array_names.append(name)
                    names_history.append(name)

        df = spark.read.parquet(*array_names)


        date_now = datetime.now(pytz.timezone('America/Bogota')).strftime("%d%m%Y_%H%M%S")
        path_write = f"/datalake/silver/stagging/{date_now}"


        df.write.parquet(path_write)

        for name in array_names:
            if ".parquet" in name:
                name_new = name.replace("stagging", "ingested")
                command = f"hadoop fs -cp {name} {name_new}"
                subprocess.run(command, shell=True, check=True)
                print(f"copiado:{name}")
                
        time.sleep(30)

except Exception as e:
    print(f"Error: {e}")
except KeyboardInterrupt:
    print("Deteniendo lectura...")


copiado:/datalake/raw/stagging/part-00000-0b1e19c2-7021-4320-b630-c32d50d2a548-c000.snappy.parquet
copiado:/datalake/raw/stagging/part-00000-2719f546-4cf1-436d-97f7-fa5f3706fa68-c000.snappy.parquet
copiado:/datalake/raw/stagging/part-00000-362ae5c9-ada2-4a03-bb93-3931cea988ce-c000.snappy.parquet
copiado:/datalake/raw/stagging/part-00000-3983f2f5-717b-49f3-8f0a-4951b03503d8-c000.snappy.parquet
copiado:/datalake/raw/stagging/part-00000-4dbe635f-098e-494e-88ec-6027c9a38798-c000.snappy.parquet
copiado:/datalake/raw/stagging/part-00000-5bb35691-ab00-4943-b080-b5d4be7b7d63-c000.snappy.parquet
copiado:/datalake/raw/stagging/part-00000-87cec15e-eda3-4b80-9cab-1ab32ec76c53-c000.snappy.parquet
copiado:/datalake/raw/stagging/part-00000-92fa1d8e-7f4e-4fdd-bf5c-2426611709d9-c000.snappy.parquet
copiado:/datalake/raw/stagging/part-00000-97e8caeb-2935-4c46-b290-3fc13bb6ff47-c000.snappy.parquet
copiado:/datalake/raw/stagging/part-00000-a079f9ed-5d32-45cc-bc8b-68c4cc9ba8ff-c000.snappy.parquet
copiado:/d